In [5]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy, precision
import torchmetrics.functional as tf

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar

from positional_encodings.torch_encodings import PositionalEncodingPermute2D

from data.cifar100 import CIFAR100DataModule
from data.cifar10 import CIFAR10DataModule

In [4]:
# Set this to whatever folder you wish CIFAR-100 to be downloaded into
CIFAR = "/media/curttigges/project-files/datasets/cifar-100/"

In [ ]:
class FourierPositionEncoding(nn.Module):

In [61]:
class PositionalImageEmbedding(nn.Module):
    """Reshapes images and concatenates position encoding
    
    Initializes position encoding,
    
    Args:

    Returns:
    
    """
    def __init__(self, input_channels, embed_dim):
        super().__init__()
        self.p_enc = PositionalEncodingPermute2D(input_channels)
        self.conv = nn.Conv1d(input_channels*2, embed_dim, 1)


    def forward(self, x):
        # initial x of shape [BATCH_SIZE x CHANNELS x HEIGHT x WIDTH]
        
        enc = p_enc(x)
        # create position encoding of the same shape as x

        x = torch.cat([x, enc], dim=1)
        # concatenate position encoding along the channel dimension
        # shape is now [BATCH_SIZE x COLOR_CHANNELS + POS_ENC_CHANNELS x HEIGHT x WIDTH]

        x = x.flatten(2)
        # reshape to [BATCH_SIZE x CHANNELS x HEIGHT*WIDTH]

        x = self.conv(x)
        # shape is now [BATCH_SIZE x EMBED_DIM x HEIGHT*WIDTH]

        x = x.permute(2, 0, 1)
        # shape is now [HEIGHT*WIDTH x BATCH_SIZE x EMBED_DIM]

        return x

In [62]:
rand_batch = torch.rand((8, 3, 224, 224))
ip = PositionalImageEmbedding(3,32)
res = ip(rand_batch)
res.shape


torch.Size([50176, 8, 32])

In [ ]:
class PerceiverCrossAttention(nn.Module):

In [ ]:
class LatentTransformer(nn.Module):

In [ ]:
class Classifier(nn.Module):

In [ ]:
class Perceiver(nn.Module):

In [ ]:
class PerceiverTrainingModule(pl.LightningModule):